# IITM RL FINAL PROJECT

## Problem - bsuite 

This notebook uses an open source reinforcement learning benchmark known as bsuite. https://github.com/deepmind/bsuite

bsuite is a collection of carefully-designed experiments that investigate core capabilities of a reinforcement learning agent.

Your task is to use any reinforcement learning techniques at your disposal to get high scores on the environments specified.

**Note**: Since the course is on Reinforcement Learning,  please limit yourself to using traditional Reinforcement Learning algorithms, 

**Do not use deep reinforcement learning.**

# How to use this notebook? 📝

- This is a shared template and any edits you make here will not be saved. **You
should make a copy in your own drive**. Click the "File" menu (top-left), then "Save a Copy in Drive". You will be working in your copy however you like.

<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/notebook/aicrowd_notebook_submission_flow.png?inline=false" alt="notebook overview" style="width: 650px;"/></p>

- **Update the config parameters**. You can define the common variables here

Variable | Description
--- | ---
`AICROWD_RESULTS_DIR` | Path to write the output to.
`AICROWD_ASSETS_DIR` | In case your notebook needs additional files (like model weights, etc.,), you can add them to a directory and specify the path to the directory here (please specify relative path). The contents of this directory will be sent to AIcrowd for evaluation.
`AICROWD_API_KEY` | In order to submit your code to AIcrowd, you need to provide your account's API key. This key is available at https://www.aicrowd.com/participants/me

- **Installing packages**. Please use the [Install packages 🗃](#install-packages-) section to install the packages

In [2]:
!pip install -q aicrowd-cli

# AIcrowd Runtime Configuration 🧷

Get login API key from https://www.aicrowd.com/participants/me


In [3]:
import os

AICROWD_RESULTS_DIR = os.getenv("OUTPUTS_DIR", "results")
os.environ["RESULTS_DIR"] = AICROWD_RESULTS_DIR
API_KEY = "6b42b562b9eb39fe9d07fb070f706ef5"

In [4]:
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


# Install packages 🗃

Please add all pacakage installations in this section

In [5]:
!pip install git+http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git
!pip install tabulate
!pip install tqdm

## Add any other installations you need here

  Cloning http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git to /tmp/pip-req-build-566f3juf
  Running command git clone -q http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git /tmp/pip-req-build-566f3juf
  Created wheel for bsuite: filename=bsuite-0.3.5-cp37-none-any.whl size=252042 sha256=eb6df203498f4cc2a273643ae3e00e9738c57acf1900f2009c994ee19780ab2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-6mj2bx_c/wheels/61/ea/06/77c82c07765fb8608e50e6c66bc566fa6d113c725bc6937e7b
Successfully built bsuite


# Import packages

In [6]:
import gym
import warnings

import numpy as np
import pandas as pd
import plotnine as gg
from tqdm.notebook import tqdm

import bsuite
from bsuite.aicrowd import environments
from bsuite.aicrowd.runner import Runner
from bsuite.aicrowd.analysis import Analyzer

import itertools
import math

pd.options.mode.chained_assignment = None
gg.theme_set(gg.theme_bw(base_size=16, base_family='serif'))
gg.theme_update(figure_size=(3, 1), panel_spacing_x=0.5, panel_spacing_y=0.5)
warnings.filterwarnings('ignore')

# **Agent Class**

You can modify the AGENT TEMPLATE below and implement the logic of your agent. Your agent must implement a few methods that will be called by the `Runner` class.
* `__init__` - put any initialization code here.
* `get_action` - takes in a `state` and returns an `action`.
* `learn` - takes in `(state, action, reward, next_state)`, implements the learning logic.
* `get_state` - takes in a raw `observation` directly from the env, discretizes it and returns a `state`.

In addition to these, you may implement other methods which can be called by the above methods.

Since there are multiple environments, you may need unique hyper parameters for each environment. Instantiate the agent while passing in the hyper parameters in a dictionary using the `agent_config` parameter so that each environment can use different hyper parameters for the agent while using a single `Agent` class for all of them.  You can use any names for the keys in the config dictionary.   

An example `RandomAgent` is given below.

In [7]:
# *** YOU CAN EDIT THIS CELL ***
# AGENT TEMPLATE
class Agent:
    def __init__(self, agent_config=None):
        self.config = agent_config

        self.action_size = self.config['env'].action_space.n
        self.env = self.config['env']

        self.Q = {}
        self.states = []

        if ('CATCH' in self.config['name']):
          self.epsilon = 0.1
          self.learning_rate = 0.6
          self.end_epsilon = self.epsilon
          self.end_learning_rate = 0.45
          self.exp_end_iter = 9000
          self.gamma = 0.9
          self.n_samples = 10000
          self.epsilon_decrement = ( self.epsilon - self.end_epsilon ) / self.exp_end_iter
          self.learning_rate_decrement = ( self.learning_rate - self.end_learning_rate ) / self.exp_end_iter
          self.catch_Q_Table_Init()
        elif ('CARTPOLE' in self.config['name']):
          self.epsilon = 1
          self.learning_rate = 0.1
          self.end_epsilon = 0.1
          self.end_learning_rate = 0.1
          self.exp_end_iter = 50
          self.gamma = 0.99
          self.epsilon_decrement = ( self.epsilon - self.end_epsilon ) / self.exp_end_iter
          self.learning_rate_decrement = ( self.learning_rate - self.end_learning_rate ) / self.exp_end_iter
          self.cartpole_Q_Table_Init()
        else:
          self.epsilon = 0.1
          self.learning_rate = 0.2
          self.gamma = 0.99
          self.weight = np.random.rand(1,(len(self.get_state(self.config['env'].reset()))+1))

    def get_action(self, state):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
        RETURNS     : 
            - action - 'action' to be taken
        '''
        if np.random.rand() <= self.epsilon:
          action =  np.random.choice(self.action_size)
        else:
          if ('MOUNTAINCAR' in self.config['name']):
            qVals = [np.matmul(self.weight,np.append(state,act))[0] for act in range(self.action_size)]
            action = qVals.index(max(qVals))
          else:  
            qVals = [self.Q[(state,act)] for act in range(self.action_size)]
            action = qVals.index(max(qVals))

        return action
    
    def learn(self, state, action, reward, next_state, done):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
            - action - 'action' performed in 'state'
            - reward - 'reward' received due to action taken
            - next_state - discretized 'next_state'
            - done - status flag to represent if an episode is done or not
        RETURNS     : 
            - NIL
        '''
        if ('MOUNTAINCAR' in self.config['name']):
          next_q_values = [np.matmul(self.weight,np.append(next_state,act))[0] for act in range(self.action_size)]
          target = reward + self.gamma * max(next_q_values)
          currQ = np.matmul(self.weight,np.append(state,action))[0]
          self.weight += ( self.learning_rate * (target - currQ) ) * np.array([np.append(state,action)])
        else:
          if (state == None) or (next_state == None):
            print(self.config['name'], "None Spotted")
            return

          next_q_values = [self.Q[(next_state,act)] for act in range(self.action_size)]
          target = reward + self.gamma * max(next_q_values)

          self.Q[(state,action)] += ( self.learning_rate * (target - self.Q[(state,action)]) )

          self.epsilon = max(self.end_epsilon, (self.epsilon - self.epsilon_decrement))
          self.learning_rate = max(self.end_learning_rate, (self.learning_rate - self.learning_rate_decrement))

    def get_state(self, observation):
        '''
        PARAMETERS  : 
            - observation - raw 'observation' from environment
        RETURNS     : 
            - state - discretized 'state' from raw 'observation'
        '''
        if ('MOUNTAINCAR' in self.config['name']):
          state = observation[0][:-1]
        else:
          if ('CATCH' in self.config['name']):
            st = self.simplify_catch_state(observation)
            if st in self.states:
              state = st
            else:
              state = None
          else:
            state = self.transform_cartpole_state(self.simplify_cartpole_state(observation))

        return state
    
    def simplify_catch_state(self, observation):
      le = list(np.where((observation[:-1])==1))
      if len(le[0]) == 0:
        i = None
      else:
        i = le[0][0]
      if len(le[1]) == 0:
        j = None
      else:
        j = le[1][0]
      ne = list(np.where((observation[-1])==1))
      if len(ne[0]) == 0:
        k = 0
      else:
        k = ne[0][0]
      return (i,j,k)

    def catch_Q_Table_Init(self):

      done = False
      s = self.simplify_catch_state(self.env.reset())
      self.states.append(s)
      for _ in tqdm(range(self.n_samples)):
        a = np.random.choice(self.action_size)
        s2, _, done, _ = self.env.step(a)
        self.states.append(self.simplify_catch_state(s2))
        if done:
          s = self.simplify_catch_state(self.env.reset())
          self.states.append(s)

      for s in self.states:
        for a in range(self.action_size):
          self.Q[(s,a)] = np.random.randn()

    def simplify_cartpole_state(self, x):
      x = x[0][:-1]
      x = [math.atan2(x[2],x[3]), x[4]]
      return x

    def transform_cartpole_state(self, state):
      x = []
      for d in range(self.D):
        x.append(int(np.digitize(state[d], self.bins[d])))
      return tuple(x)

    def cartpole_Q_Table_Init(self):
      n_bins = 13
      max = np.array([[ 1., 5., 1., 1., 5., 1.]])
      min = np.array([[-1., -5., -1., -1., -5., 0.]])
      car_max = self.simplify_cartpole_state(max)
      #print(car_max)
      car_min = self.simplify_cartpole_state(min)
      #print(car_min)
      self.D = len(car_max)
      self.bins = []
      for _ in range(self.D):
        self.bins.append(np.linspace(car_min[_], car_max[_], num=n_bins))
      list_of_bins = []
      for d in range(self.D):
        list_of_bins.append(list(range(len(self.bins[d]) + 1)))
      for s in itertools.product(*list_of_bins):
        for a in range(self.action_size):
          self.Q[(s,a)] = 0

In [8]:
# *** YOU CAN EDIT THIS CELL ***
# DO NOT rename the config dictionaries as the evaluator references them. However, you may use any names for the keys in them.
catch_config = {'env':environments.load_env(environments.CATCH),'name':'CATCH'}
catch_noise_config = {'env':environments.load_env(environments.CATCH_NOISE),'name':'CATCH_NOISE'}
cartpole_config = {'env':environments.load_env(environments.CARTPOLE),'name':'CARTPOLE'}
cartpole_noise_config = {'env':environments.load_env(environments.CARTPOLE_NOISE),'name':'CARTPOLE_NOISE'}
mountaincar_config = {'env':environments.load_env(environments.MOUNTAINCAR),'name':'MOUNTAINCAR'}
mountaincar_noise_config = {'env':environments.load_env(environments.MOUNTAINCAR_NOISE),'name':'MOUNTAINCAR_NOISE'}

Loaded bsuite_id: catch/0.
Loaded bsuite_id: catch_noise/1.
Loaded bsuite_id: cartpole/0.
Loaded bsuite_id: cartpole_noise/1.
Loaded bsuite_id: mountain_car/0.
Loaded bsuite_id: mountain_car_noise/1.


In [9]:
# *** YOU CAN EDIT THIS CELL ***
# EXAMPLE
class RandomAgent:
    def __init__(self, agent_config={}):
        self.config = agent_config
        self.env_name = self.config['env_name']

    def get_action(self, state):
        action = np.random.choice(2)
        return action
    
    def learn(self, state, action, reward, next_state, done):
        if ('BAR' in self.config):
            if (self.config['BAR']):
                self.config['FOO'] += 1

    def get_state(self, observation):
        # In this function you're allowed to use 
        # the environment name for observation preprocessing
        # Do not use it anywhere else
        if self.env_name == 'catch':
          state = observation
        elif self.env_name == 'catch_noise':
          state = observation
        elif self.env_name == 'cartpole':
          state = observation
        elif self.env_name == 'cartpole_noise':
          state = observation
        elif self.env_name == 'mountaincar':
          state = observation
        elif self.env_name == 'mountaincar_noise':
          state = observation
        else:
          raise NotImplementedError

        return state

env1_config = {
    "env_name": 'cartpole',
    'FOO': 0.1,
    'BAR': True
}

env2_config = {
    "env_name": 'cartpole',
    'FOO': 0.2,
    'BAR': False
}

randomAgent1 = RandomAgent(agent_config=env1_config)
randomAgent2 = RandomAgent(agent_config=env2_config)

# **Playing with the Environment**

#### **Instantiating the environment** :
You can create an environment by calling the following function:  
`environments.load_env(ENV_ID)` - RETURNS: `env`  
where, ENV_ID can be ONE of the following:
* `environments.CATCH`
* `environments.CATCH_NOISE`
* `environments.CARTPOLE`
* `environments.CARTPOLE_NOISE`
* `environments.MOUNTAINCAR`
* `environments.MOUNTAINCAR_NOISE`

The `NOISE` environments add a scaled random noise to the `reward`.
<br/>

#### **Runnning the environment** :
There are certain methods required to run the environments. The interface is very similar to OpenAI Gym's interfaces. Fore more information, read the OpenAI documentation [here](https://gym.openai.com/docs/).

`env.reset()` - RETURNS: `observation`  
`env.step(action)`  - RETURNS: `(next_observation, reward, done, info[NOT USED])`

There are also a few useful properties within the environments:

* `env.action_space.n` - total number of possible actions. eg: if 'n' is 3, then the possible actions are `[0, 1, 2]`
* `env.observation_space.shape` -  the shape of the observation.
* `env.bsuite_num_episodes` -  the pre-specified number of episodes which will be run during evaluation (unique for each environment).

##### *ONLY IN CATCH / CATCH_NOISE*
* `env.observation_space.high` -  the upper limit for every index in the observation.
* `env.observation_space.low` -  the lower limit for every index of the observation.
<br/>


## **Environment Observation Space Limits:**

The limits for the observation space (minimum and maximum) for all the environments are given in the table below:

| Environments                        | Limits                                                                      |
|-------------------------------------|-----------------------------------------------------------------------------|
| CATCH <br/>  CATCH_NOISE            | MIN: use `env.observation_space.low` <br/> MAX: use `env.observation_space.high` |
| CARTPOLE <br/> CARTPOLE_NOISE       | MIN: `[-1. -5., -1., -1., -5., 0.]` <br/> MAX: `[ 1.,  5.,  1.,  1.,  5., 1.]` |
| MOUNTAINCAR <br/> MOUNTAINCAR_NOISE | MIN: `[-1.2, -0.07, 0.]` <br/> MAX: `[ 0.6,  0.07,  1.]`                                 |

[NOTE] Use this code cell to play around and get used to the environments. However, the `Runner` class below will be used to evaluate your agent.

In [10]:
# *** YOU CAN EDIT THIS CELL ***
# TEST AREA
env = environments.load_env(environments.CARTPOLE)  # replace 'environments.CARTPOLE' with other environments

agent = RandomAgent(agent_config={"env_name": 'cartpole'})    # replace with 'RandomAgent()' to use your custom agent

NUM_EPISODES = 10                                   # replace with 'env.bsuite_num_episodes' to run for pre-specified number of episodes
for episode_n in tqdm(range(NUM_EPISODES)):
    done = False
    episode_reward = 0
    episode_moves = 0
 
    observation = env.reset()
    state = agent.get_state(observation)

    while not done:
        action = agent.get_action(state)

        next_observation, reward, done, _ = env.step(action)
        next_state = agent.get_state(next_observation)

        agent.learn(state, action, reward, next_state, done)

        state = next_state

        episode_reward += reward
        episode_moves += 1

    if (((episode_n+1) % 2) == 0): 
        print("EPISODE: ",episode_n+1,"\tREWARD: ",episode_reward,"\tEPISODE_LENGTH: ",episode_moves)

Loaded bsuite_id: cartpole/0.


  0%|          | 0/10 [00:00<?, ?it/s]

EPISODE:  2 	REWARD:  34.0 	EPISODE_LENGTH:  35
EPISODE:  4 	REWARD:  39.0 	EPISODE_LENGTH:  40
EPISODE:  6 	REWARD:  38.0 	EPISODE_LENGTH:  39
EPISODE:  8 	REWARD:  39.0 	EPISODE_LENGTH:  40
EPISODE:  10 	REWARD:  34.0 	EPISODE_LENGTH:  35


## Point to the Agent Class you'll use for the final score

In [11]:
RLAgent = Agent

# **Evaluating the Agent on all the Environments**

* The following cells will take care of running your agent on each environment and aggregating the results in csv files. In each of the following cells, the `agent_config` parameter is already set to use the corresponding config dictionary for that environment. DO NOT EDIT THIS.
* Feel free to modify the `LOG_INTERVAL` parameter to change the interval between episodes for logging.  
* Please do not modify any other contents in each of the cells.  

In [12]:
LOG_INTERVAL = 100

In [13]:
runner = Runner(
    agent = RLAgent(agent_config=catch_config),
    env_id = environments.CATCH,
    log_interval = LOG_INTERVAL,
)
runner.play_episodes()

  0%|          | 0/10000 [00:00<?, ?it/s]

Loaded bsuite_id: catch/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/10000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  1.0 	MEAN_REWARD:  -0.58 	EPISODE_LENGTH:  9
EPISODE:  200 	REWARD:  -1.0 	MEAN_REWARD:  0.08 	EPISODE_LENGTH:  9
EPISODE:  300 	REWARD:  1.0 	MEAN_REWARD:  0.36 	EPISODE_LENGTH:  9
EPISODE:  400 	REWARD:  -1.0 	MEAN_REWARD:  0.38 	EPISODE_LENGTH:  9
EPISODE:  500 	REWARD:  1.0 	MEAN_REWARD:  0.44 	EPISODE_LENGTH:  9
EPISODE:  600 	REWARD:  -1.0 	MEAN_REWARD:  0.44 	EPISODE_LENGTH:  9
EPISODE:  700 	REWARD:  -1.0 	MEAN_REWARD:  0.38 	EPISODE_LENGTH:  9
EPISODE:  800 	REWARD:  1.0 	MEAN_REWARD:  0.34 	EPISODE_LENGTH:  9
EPISODE:  900 	REWARD:  1.0 	MEAN_REWARD:  0.48 	EPISODE_LENGTH:  9
EPISODE:  1000 	REWARD:  1.0 	MEAN_REWARD:  0.42 	EPISODE_LENGTH:  9
EPISODE:  1100 	REWARD:  1.0 	MEAN_REWARD:  0.32 	EPISODE_LENGTH:  9
EPISODE:  1200 	REWARD:  1.0 	MEAN_REWARD:  0.36 	EPISODE_LENGTH:  9
EPISODE:  1300 	REWARD:  1.0 	MEAN_REWARD:  0.38 	EPISODE_LENGTH:  9
EPISODE:  1400 	REWARD:  1.0 	MEAN_REWARD:  0.44 	EPISODE_LENGTH:  9
EPISODE:  1500 	REWARD:  -1.0 	MEAN_RE

In [14]:
runner = Runner(
    agent = RLAgent(agent_config=catch_noise_config),
    env_id = environments.CATCH_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

  0%|          | 0/10000 [00:00<?, ?it/s]

Loaded bsuite_id: catch_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/10000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -2.4319180485080194 	MEAN_REWARD:  0.35 	EPISODE_LENGTH:  9
EPISODE:  200 	REWARD:  2.888194910660849 	MEAN_REWARD:  0.76 	EPISODE_LENGTH:  9
EPISODE:  300 	REWARD:  0.7058772924254098 	MEAN_REWARD:  0.61 	EPISODE_LENGTH:  9
EPISODE:  400 	REWARD:  0.5749511984405982 	MEAN_REWARD:  0.82 	EPISODE_LENGTH:  9
EPISODE:  500 	REWARD:  1.8165310035839968 	MEAN_REWARD:  0.87 	EPISODE_LENGTH:  9
EPISODE:  600 	REWARD:  0.39936420173707576 	MEAN_REWARD:  0.77 	EPISODE_LENGTH:  9
EPISODE:  700 	REWARD:  1.3884776851441103 	MEAN_REWARD:  0.67 	EPISODE_LENGTH:  9
EPISODE:  800 	REWARD:  1.8585504709523746 	MEAN_REWARD:  0.74 	EPISODE_LENGTH:  9
EPISODE:  900 	REWARD:  1.362278430457311 	MEAN_REWARD:  0.93 	EPISODE_LENGTH:  9
EPISODE:  1000 	REWARD:  1.829857544425355 	MEAN_REWARD:  0.85 	EPISODE_LENGTH:  9
EPISODE:  1100 	REWARD:  -2.373967525284758 	MEAN_REWARD:  0.77 	EPISODE_LENGTH:  9
EPISODE:  1200 	REWARD:  0.9410664532715053 	MEAN_REWARD:  0.72 	EPISODE_LENGTH:  9
EP

In [15]:
runner = Runner(
    agent = RLAgent(agent_config=cartpole_config),
    env_id = environments.CARTPOLE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: cartpole/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  117.0 	MEAN_REWARD:  95.12 	EPISODE_LENGTH:  118
EPISODE:  200 	REWARD:  170.0 	MEAN_REWARD:  123.41 	EPISODE_LENGTH:  171
EPISODE:  300 	REWARD:  191.0 	MEAN_REWARD:  137.94 	EPISODE_LENGTH:  192
EPISODE:  400 	REWARD:  169.0 	MEAN_REWARD:  164.24 	EPISODE_LENGTH:  170
EPISODE:  500 	REWARD:  592.0 	MEAN_REWARD:  243.92 	EPISODE_LENGTH:  593
EPISODE:  600 	REWARD:  1001.0 	MEAN_REWARD:  597.75 	EPISODE_LENGTH:  1001
EPISODE:  700 	REWARD:  1001.0 	MEAN_REWARD:  948.26 	EPISODE_LENGTH:  1001
EPISODE:  800 	REWARD:  1001.0 	MEAN_REWARD:  964.29 	EPISODE_LENGTH:  1001
EPISODE:  900 	REWARD:  1001.0 	MEAN_REWARD:  903.73 	EPISODE_LENGTH:  1001
EPISODE:  1000 	REWARD:  1001.0 	MEAN_REWARD:  970.07 	EPISODE_LENGTH:  1001


In [16]:
runner = Runner(
    agent = RLAgent(agent_config=cartpole_noise_config),
    env_id = environments.CARTPOLE_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: cartpole_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  47.79511901113902 	MEAN_REWARD:  80.68 	EPISODE_LENGTH:  48
EPISODE:  200 	REWARD:  133.72600621528264 	MEAN_REWARD:  96.79 	EPISODE_LENGTH:  136
EPISODE:  300 	REWARD:  415.88311823384976 	MEAN_REWARD:  251.07 	EPISODE_LENGTH:  412
EPISODE:  400 	REWARD:  365.0789386832645 	MEAN_REWARD:  308.16 	EPISODE_LENGTH:  370
EPISODE:  500 	REWARD:  673.681359074358 	MEAN_REWARD:  390.06 	EPISODE_LENGTH:  676
EPISODE:  600 	REWARD:  98.78778973586526 	MEAN_REWARD:  440.06 	EPISODE_LENGTH:  99
EPISODE:  700 	REWARD:  369.73349093958115 	MEAN_REWARD:  389.98 	EPISODE_LENGTH:  374
EPISODE:  800 	REWARD:  87.92539500608397 	MEAN_REWARD:  455.31 	EPISODE_LENGTH:  87
EPISODE:  900 	REWARD:  76.61241739316138 	MEAN_REWARD:  395.03 	EPISODE_LENGTH:  79
EPISODE:  1000 	REWARD:  93.37109738184989 	MEAN_REWARD:  372.61 	EPISODE_LENGTH:  92


In [17]:
runner = Runner(
    agent = RLAgent(agent_config=mountaincar_config),
    env_id = environments.MOUNTAINCAR,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: mountain_car/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -225.0 	MEAN_REWARD:  -315.03 	EPISODE_LENGTH:  225
EPISODE:  200 	REWARD:  -209.0 	MEAN_REWARD:  -242.8 	EPISODE_LENGTH:  209
EPISODE:  300 	REWARD:  -164.0 	MEAN_REWARD:  -190.14 	EPISODE_LENGTH:  164
EPISODE:  400 	REWARD:  -1000.0 	MEAN_REWARD:  -284.4 	EPISODE_LENGTH:  1000
EPISODE:  500 	REWARD:  -129.0 	MEAN_REWARD:  -259.79 	EPISODE_LENGTH:  129
EPISODE:  600 	REWARD:  -164.0 	MEAN_REWARD:  -311.41 	EPISODE_LENGTH:  164
EPISODE:  700 	REWARD:  -260.0 	MEAN_REWARD:  -284.29 	EPISODE_LENGTH:  260
EPISODE:  800 	REWARD:  -144.0 	MEAN_REWARD:  -272.01 	EPISODE_LENGTH:  144
EPISODE:  900 	REWARD:  -139.0 	MEAN_REWARD:  -274.06 	EPISODE_LENGTH:  139
EPISODE:  1000 	REWARD:  -121.0 	MEAN_REWARD:  -343.81 	EPISODE_LENGTH:  121


In [18]:
runner = Runner(
    agent = RLAgent(agent_config=mountaincar_noise_config),
    env_id = environments.MOUNTAINCAR_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: mountain_car_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -133.01292243870157 	MEAN_REWARD:  -302.31 	EPISODE_LENGTH:  133
EPISODE:  200 	REWARD:  -189.3194924907309 	MEAN_REWARD:  -223.86 	EPISODE_LENGTH:  185
EPISODE:  300 	REWARD:  -202.1475321990124 	MEAN_REWARD:  -189.78 	EPISODE_LENGTH:  203
EPISODE:  400 	REWARD:  -330.48273523469936 	MEAN_REWARD:  -223.34 	EPISODE_LENGTH:  325
EPISODE:  500 	REWARD:  -136.5803974066393 	MEAN_REWARD:  -245.47 	EPISODE_LENGTH:  138
EPISODE:  600 	REWARD:  -121.41296328352249 	MEAN_REWARD:  -287.24 	EPISODE_LENGTH:  119
EPISODE:  700 	REWARD:  -125.10165174564982 	MEAN_REWARD:  -317.61 	EPISODE_LENGTH:  125
EPISODE:  800 	REWARD:  -143.99016315452153 	MEAN_REWARD:  -292.26 	EPISODE_LENGTH:  144
EPISODE:  900 	REWARD:  -238.5085707683429 	MEAN_REWARD:  -293.22 	EPISODE_LENGTH:  238
EPISODE:  1000 	REWARD:  -319.92800067561416 	MEAN_REWARD:  -279.99 	EPISODE_LENGTH:  315


# **Analysis & Result**

The following cells will show the score of the agent on each environment. The same scoring method will be used to evaluate your agent on a set of test environments.

In [19]:
# *** PLEASE DONT EDIT THE CONTENTS OF THIS CELL ***
analyzer = Analyzer(os.environ.get('RESULTS_DIR'))
analyzer.print_scores()

╒════════════════════╤══════════╕
│ ENVIRONMENT        │    SCORE │
╞════════════════════╪══════════╡
│ catch              │ 0.65775  │
├────────────────────┼──────────┤
│ catch_noise        │ 0.85175  │
├────────────────────┼──────────┤
│ cartpole           │ 0.757437 │
├────────────────────┼──────────┤
│ cartpole_noise     │ 0.658921 │
├────────────────────┼──────────┤
│ mountain_car       │ 0.822226 │
├────────────────────┼──────────┤
│ mountain_car_noise │ 0.834349 │
╘════════════════════╧══════════╛


In [20]:
# If you want a object to get the scores
analyzer.get_scores()

{'cartpole': 0.7574365000000001,
 'cartpole_noise': 0.6589205,
 'catch': 0.65775,
 'catch_noise': 0.85175,
 'mountain_car': 0.822226,
 'mountain_car_noise': 0.8343489999999999}

## What is the score function

The score function is developed by the BSuite team at Deepmind. It is open source and available at https://github.com/deepmind/bsuite

The score measures behavioral aspects of the agent only, and does not take into account internal state of the agent. For more details read Section 2 of the [BSuite paper](https://openreview.net/forum?id=rygf-kSYwH). In this case we use only the "Basic" aspect of the agent's scoring system.

**It is not necessary to understand the score in order to improve your agent's performance**

# **Backend Evaluation**

THIS CODE WILL EVALUATE THE AGENT USING THE SPECIFIED CONFIGS FOR THE CORRESPONDING ENVIRONMENTS. DO NOT EDIT THE CONTENTS OF THIS CELL.

In [21]:
## Do not edit this cell
if (os.environ.get('BACKEND_EVALUATOR') is not None):
    
    import backend_evaluator

    runs = {
        'catch': (
            backend_evaluator.CATCH, 
            catch_config),
        'catch_noise': (
            backend_evaluator.CATCH_NOISE, 
            catch_noise_config),
        'cartpole': (
            backend_evaluator.CARTPOLE, 
            cartpole_config),
        'cartpole_noise': (
            backend_evaluator.CARTPOLE_NOISE, 
            cartpole_noise_config),
        'mountaincar': (
            backend_evaluator.MOUNTAINCAR, 
            mountaincar_config),
        'mountaincar_noise': (
            backend_evaluator.MOUNTAINCAR_NOISE, 
            mountaincar_noise_config)
    }

    for run_name, run in runs.items():
        env_ids, config = run
        for env_id in env_ids:
            runner = Runner(env_id=env_id,
                            agent=RLAgent(agent_config=config),
                            verbose=False,
                            eval=True)
            runner.play_episodes()

# Submit to AIcrowd 🚀

**NOTE: PLEASE SAVE THE NOTEBOOK BEFORE SUBMITTING IT (Ctrl + S)**

In [22]:
! aicrowd notebook submit --no-verify -c iitm-rl-final-project -a assets

Mounting Google Drive 💾
Your Google Drive will be mounted to access the colab notebook
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g7BDso6hU2kLcA-BSWB4xgJTD45MywoklO0D8PtTorncVUAwWguiy0
Mounted at /content/drive
Using notebook: /content/drive/MyDrive/Colab Notebooks/Copy of Copy of Copy of bsuite-challenge-starter-kit[COPY].ipynb for submissio